In [2]:
import cv2
import tkinter as tk
import tkinter.filedialog as tkfd
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import  Flatten, Dense, Activation,Convolution2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import backend as K
import random
from keras.utils import Sequence, multi_gpu_model

In [3]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_val = pd.read_csv('./data/val.csv')

In [4]:
folder = './data/preprocessed'

X_train = []
y_train = []
X_test = []
y_test = []
X_val = []
y_val = []

for filename in df_train['image_id']:
    filename += '.jpg'
    img=cv2.imread(os.path.join(folder,filename))
    X_train.append(img)

y_train = df_train.iloc[:, 2:].values

In [5]:
X_train = np.array(X_train)

In [6]:
X_train.shape

(1200, 256, 256, 3)

In [7]:
y_train.shape

(1200, 3)

In [8]:
y_train[0]

array([0, 0, 1])

In [9]:
for filename in df_test['image_id']:
    filename += '.jpg'
    img=cv2.imread(os.path.join(folder,filename))
    X_test.append(img)

y_test = df_test.iloc[:, 2:].values
X_test = np.array(X_test)

In [10]:
X_test.shape

(400, 256, 256, 3)

In [11]:
y_test.shape

(400, 3)

In [12]:
for filename in df_val['image_id']:
    filename += '.jpg'
    img=cv2.imread(os.path.join(folder,filename))
    X_val.append(img)

y_val = df_val.iloc[:, 2:].values
X_val = np.array(X_val)

In [13]:
X_val.shape

(400, 256, 256, 3)

In [14]:
y_val.shape

(400, 3)

In [20]:
class MiniBatchGenerator(Sequence):

    def __init__(self, X, y) :
        self.X = X
        self.y = y
        self.batch_size = 16

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, index):
        self.input = []
        self.output = []

        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, self.X.shape[0])
        pos = 0
        for i in range(start, end):
            self.input.append(self.X[i])
            self.output.append(self.y[i])
            pos += 1

        return np.array(self.input), np.array(self.output)

In [21]:
train_gen = MiniBatchGenerator(X_train, y_train)
val_gen = MiniBatchGenerator(X_val, y_val)

## InceptionV3

In [23]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD

base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

#Uncomment below line if training on multiple GPUs (distributed environment)
G = 2 #Number of gpus
#model = multi_gpu_model(model, gpus=G)
    
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

print("Started training...")
EPOCHS = 20
history = model.fit_generator(generator=train_gen,
                                  validation_data=val_gen,
                                  epochs=EPOCHS,
                                  use_multiprocessing=True)

Started training...
Epoch 1/20
 5/75 [=>............................] - ETA: 4:12 - loss: 1.2007

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/pyt

Epoch 1/20
Epoch 1/20


KeyboardInterrupt: 

## ResNet

In [25]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:30]:
    layer.trainable = False

for layer in model.layers[30:]:
    layer.trainable = True

#Uncomment below line if training on multiple GPUs (distributed environment)
G = 2 #Number of gpus
#model = multi_gpu_model(model, gpus=G)
    
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

print("Started training...")
EPOCHS = 20
history = model.fit_generator(generator=train_gen,
                                  validation_data=val_gen,
                                  epochs=EPOCHS,
                                  use_multiprocessing=True)

Started training...
Epoch 1/20
 1/75 [..............................] - ETA: 27:00 - loss: 1.4403

Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/py

Epoch 1/20


KeyboardInterrupt


Epoch 1/20


KeyboardInterrupt: 

## VGG19

In [26]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True

#Uncomment below line if training on multiple GPUs (distributed environment)
G = 2 #Number of gpus
#model = multi_gpu_model(model, gpus=G)
    
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

print("Started training...")
EPOCHS = 20
history = model.fit_generator(generator=train_gen,
                                  validation_data=val_gen,
                                  epochs=EPOCHS,
                                  use_multiprocessing=True)

/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Started training...
Epoch 1/20
 2/75 [..............................] - ETA: 22:07 - loss: 1.0130

Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/anandzutshi/.pyenv/versions/3.6.7/lib/p

Epoch 1/20
Epoch 1/20


KeyboardInterrupt: 